In [29]:
from sklearn.linear_model import LinearRegression
import pandas as pd

df = pd.read_csv('World-Stock-Prices-Dataset.csv')


df["Country"].unique()


array(['usa', 'germany', 'japan', 'switzerland', 'canada', 'netherlands',
       'france'], dtype=object)

In [30]:
# Explore the dataset structure
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nBasic statistics:")
print(df.describe())


Dataset shape: (310122, 13)

Column names:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Brand_Name', 'Ticker', 'Industry_Tag', 'Country', 'Dividends', 'Stock Splits', 'Capital Gains']

First few rows:
                        Date        Open        High         Low       Close  \
0  2025-07-03 00:00:00-04:00    6.630000    6.740000    6.615000    6.640000   
1  2025-07-03 00:00:00-04:00  106.750000  108.370003  106.330101  107.339996   
2  2025-07-03 00:00:00-04:00  122.629997  123.050003  121.550003  121.930000   
3  2025-07-03 00:00:00-04:00  221.705002  224.009995  221.360001  223.410004   
4  2025-07-03 00:00:00-04:00  212.145004  214.649994  211.810104  213.550003   

       Volume Brand_Name Ticker Industry_Tag  Country  Dividends  \
0   4209664.0    peloton   PTON      fitness      usa        0.0   
1    560190.0      crocs   CROX     footwear      usa        0.0   
2     36600.0     adidas  ADDYY      apparel  germany        0.0   
3  29295154.0     amazon   AMZN   e-com

In [31]:
#questions 1-3
#1
a=df.sort_values(by="High",ascending=False)
print(a.head(1))
#2
d={}
d2={}
for x,y in df.iterrows():
    # print(y[6])
    if y[6] in d:
        d[y[6]][0]+=y[5]
        d[y[6]][1]+=1
    else:
        d[y[6]]=[y[5],1]
for x in d.keys():
    d2[x]=(d[x][0]/d[x][1])
print(d2)
#3
d3={}
for x,y in df.iterrows():
    if y[9] in d3:
        d3[y[9]]+=1
    else:
        d3[y[9]]=1
print(d3)

                            Date    Open         High     Low        Close  \
18924  2024-06-18 00:00:00-04:00  3435.0  3463.070068  3370.0  3427.610107   

         Volume Brand_Name Ticker Industry_Tag Country  Dividends  \
18924  693800.0   chipotle    CMG         food     usa        0.0   

       Stock Splits  Capital Gains  
18924           0.0            NaN  


/var/folders/h8/c23d2rp519z5xwqk6hjrz6gm0000gp/T/ipykernel_21779/637325651.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if y[6] in d:
/var/folders/h8/c23d2rp519z5xwqk6hjrz6gm0000gp/T/ipykernel_21779/637325651.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d[y[6]]=[y[5],1]
/var/folders/h8/c23d2rp519z5xwqk6hjrz6gm0000gp/T/ipykernel_21779/637325651.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d[y[6]][0]+=y[5]
/var

KeyboardInterrupt: 

4) which company had the most volatile stock in terms of daily range (high to low gap)

In [3]:
df['Volatility (%)'] = (df['High'] - df['Low']) / df['Low'] * 100
max_vol_row = df.loc[df['Volatility (%)'].idxmax()]
print(max_vol_row)

Date              2019-06-10 00:00:00-04:00
Open                             590.928222
High                             600.041075
Low                               67.861498
Close                             92.097748
Volume                                200.0
Brand_Name                             puma
Ticker                                PMMAF
Industry_Tag                        apparel
Country                             germany
Dividends                               0.0
Stock Splits                           10.0
Capital Gains                           NaN
Volatility (%)                   784.214303
Name: 94646, dtype: object


5) identify the companies that paid dividends at some point in the dataset

In [46]:
df[df["Dividends"] > 0]["Brand_Name"].unique()

In [ ]:
#question 7
d4={}
print(df["Brand_Name"].unique())
for x in df["Brand_Name"].unique():
    d4[x]=["","",0]
# print(df["Brand_Name"].unique()[0])
mx=0
for i in df["Brand_Name"].unique():
    l=[]
    for x,y in df.iterrows():
        # print(i,y[6])
        if y[6]!=i:
            continue
        l.append([y[0],y[4]])
    # print(sorted(l,key=lambda x: x[1],reverse=True))
    # break
    for x in range(len(l)-5):
        a=l[x][0]
        b=l[x+5][0]
        cnt=0
        for y in range(5):
            cnt+=l[x+y][1]
        # print(d4[i][2])
        mx=max(mx,cnt)
        if d4[i][2]<=cnt:
            d4[i][2]=cnt
            d4[i][0]=a
            d4[i][1]=b
    # print(mx)
    # break
print(d4)

['peloton' 'crocs' 'adidas' 'amazon' 'apple' 'nike' 'target' 'google'
 'spotify' 'zoom video communications' 'the walt disney company' 'roblox'
 'delta air lines' 'costco' 'southwest airlines'
 'american eagle outfitters' 'tesla' 'starbucks' 'nvidia'
 'salesforce / slack' 'honda' 'colgate palmolive' 'hershey company'
 'chipotle' 'pinterest' 'logitech' 'shopify' 'amd' 'american express'
 'coinbase' 'mastercard' "mcdonald's" 'adobe' 'unilever' 'cisco'
 'jpmorgan chase & co' 'airbnb' 'marriott' 'toyota' 'hilton'
 'the home depot' 'johnson & johnson' 'uber' 'procter & gamble' 'fedex'
 '3m' 'philips' 'foot locker' 'the coca-cola company' 'microsoft' 'visa'
 'lvmh' 'zoominfo' 'ubisoft' 'netflix' 'puma' 'nintendo' 'bmw group'
 'porsche' 'twitter' 'nordstrom' 'block']


/var/folders/h8/c23d2rp519z5xwqk6hjrz6gm0000gp/T/ipykernel_21779/3554267303.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if y[6]!=i:
/var/folders/h8/c23d2rp519z5xwqk6hjrz6gm0000gp/T/ipykernel_21779/3554267303.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  l.append([y[0],y[4]])


799.9299926757812
{'peloton': ['2021-01-14 00:00:00-05:00', '2021-01-07 00:00:00-05:00', 799.9299926757812], 'crocs': ['', '', 0], 'adidas': ['', '', 0], 'amazon': ['', '', 0], 'apple': ['', '', 0], 'nike': ['', '', 0], 'target': ['', '', 0], 'google': ['', '', 0], 'spotify': ['', '', 0], 'zoom video communications': ['', '', 0], 'the walt disney company': ['', '', 0], 'roblox': ['', '', 0], 'delta air lines': ['', '', 0], 'costco': ['', '', 0], 'southwest airlines': ['', '', 0], 'american eagle outfitters': ['', '', 0], 'tesla': ['', '', 0], 'starbucks': ['', '', 0], 'nvidia': ['', '', 0], 'salesforce / slack': ['', '', 0], 'honda': ['', '', 0], 'colgate palmolive': ['', '', 0], 'hershey company': ['', '', 0], 'chipotle': ['', '', 0], 'pinterest': ['', '', 0], 'logitech': ['', '', 0], 'shopify': ['', '', 0], 'amd': ['', '', 0], 'american express': ['', '', 0], 'coinbase': ['', '', 0], 'mastercard': ['', '', 0], "mcdonald's": ['', '', 0], 'adobe': ['', '', 0], 'unilever': ['', '', 0], 